In [6]:
import subprocess
import pandas as pd
import os
import joblib
import shutil

In [7]:
java_code_folder = "C:/Users/rachi/OneDrive/Desktop/MyFolder/CodeSmell - 3/data/java"

ck_jar_path = 'ck-0.7.1.jar'
ck_output_folder = 'ck_output'
combined_dataset_file = 'combined_code_metrics.csv'
pipeline_path = 'output/code_smell_preprocessing_pipeline.joblib'
features_path = 'output/selected_features.joblib'
model_path = 'output/code_smell_detection_model.joblib'

In [8]:
print("Running CK tool to extract metrics...")
os.makedirs(ck_output_folder, exist_ok=True)

ck_command = [
    'java',
    '-jar',
    os.path.join("..", ck_jar_path),
    java_code_folder,
    'false',
    '0',
    'false'
]

try:
    subprocess.run(ck_command, check=True, capture_output=True, text=True, cwd=ck_output_folder)
    print(f"CK tool finished. Output files saved in: {ck_output_folder}")
except subprocess.CalledProcessError as e:
    print(f"Error running CK tool: {e}")
    print(f"Stdout: {e.stdout}")
    print(f"Stderr: {e.stderr}")
    os.path.exists(ck_output_folder) and shutil.rmtree(ck_output_folder)

Running CK tool to extract metrics...
CK tool finished. Output files saved in: ck_output


In [9]:
code_labels = {
    "file_3763661.java": 1,
    "file_5394068.java": 1,
    "file_8347642.java": 1,
    "file_5991953.java": 1,
    "file_3727271.java": 1,
    "file_4558704.java": 1,
    "file_4933806.java": 0,
    "file_5766093.java": 0,
    "file_8497585.java": 0,
    "file_5668440.java": 0,
    "file_7280795.java": 0,
    "file_4129438.java": 0,
    "file_8008116.java": 0,
    "file_4794099.java": 0,
    "file_4435545.java": 0
}


In [10]:
print("Running Code Smell Detection...", end="\n\n")

try:
    loaded_pipeline = joblib.load(pipeline_path)
    selected_features = joblib.load(features_path)
    loaded_model = joblib.load(model_path)

    new_data = pd.read_csv(os.path.join(ck_output_folder, "class.csv"))

    missing_features = [feature for feature in selected_features if feature not in new_data.columns]
    if missing_features:
        print(f"Error: The following selected features are missing in the new data: {missing_features}")
        print("Please ensure the CK output contains the necessary metrics.")

    new_data_selected = new_data[selected_features]
    new_data_processed = loaded_pipeline.transform(new_data_selected)
    predictions = loaded_model.predict(new_data_processed)

    correct, total = 0, 0
    
    print(f"Code Smell Predictions for the Java classes ({java_code_folder}):")
    for index, prediction in enumerate(predictions):
        class_name = new_data['class'].iloc[index]
        file = os.path.basename(new_data['file'].iloc[index])
        actual = code_labels[file]
        if prediction==actual:
            correct += 1
        total += 1
        print(f"File: {file}, Class: {class_name}, Predicted Label: {prediction}, Actual Label: {actual}")
    
    print(f"Correct {correct}/{total} = {int(correct/total*100)}%")

except FileNotFoundError as e:
    print(f"Error: Could not find saved model or features: {e.filename}")
except Exception as e:
    print(f"An error occurred during prediction: {e}")
finally:
    os.path.exists(ck_output_folder) and shutil.rmtree(ck_output_folder)

Running Code Smell Detection...

Code Smell Predictions for the Java classes (C:/Users/rachi/OneDrive/Desktop/MyFolder/CodeSmell - 3/data/java):
File: file_4933806.java, Class: LocaleComparator, Predicted Label: 0, Actual Label: 0
File: file_5668440.java, Class: RdfFreemarkerCli, Predicted Label: 0, Actual Label: 0
File: file_5991953.java, Class: ShapeMarshaller, Predicted Label: 1, Actual Label: 1
File: file_7280795.java, Class: ShopPolicy, Predicted Label: 0, Actual Label: 0
File: file_8497585.java, Class: VmDiskStatisticsDaoImpl, Predicted Label: 0, Actual Label: 0
File: file_4435545.java, Class: BacklogMetricBean, Predicted Label: 0, Actual Label: 0
File: file_4558704.java, Class: TemplateConfiguration, Predicted Label: 1, Actual Label: 1
File: file_5394068.java, Class: MetricsIndexerSource, Predicted Label: 1, Actual Label: 1
File: file_3763661.java, Class: SetOrderReferenceDetailsRequest, Predicted Label: 1, Actual Label: 1
File: file_8347642.java, Class: Callout, Predicted Label